In [2]:
import pandas as pd
df = pd.read_excel('fanpage_clean.xlsx')

In [3]:
df.head(1)

,id,context,time,shares,likes,love,wow,haha,angry,sad,comments,likes_count,curator,hour,weekday
0,124616330906800_1560501197318299,阿娘威！披羊皮的狼？竟大口嚼小雞\n#要打統編：小編真的是快嚇死了...😬😬\n\n影片來源...,2017-06-05 11:09:40,560,1144,11,757,21,292,86,589,2311,要打統編,11,Monday


# Text mining

In [4]:
import nltk
import jieba
from nltk.corpus import stopwords
from tqdm import tqdm, tqdm_notebook
jieba.set_dictionary('/Users/andy/Documents/python/校外課程&自我學習/jieba//dict.txt.big.txt')
stop=stopwords.words('/Users/andy/Documents/python/校外課程&自我學習/jieba/stop_words2.txt')

進行斷字斷詞以及加入停止詞stopwords

In [5]:
jieba_text = []
for index in tqdm_notebook(range(len(df))):
    words = jieba.cut(str(df['context'][index]))
    text, text2 = [], ''
    for word in words:
        if word not in stop:
            text.append(word)
            text2 += " "+word
    jieba_text.append([text,text2,len(text)])

Building prefix dict from /Users/andy/Documents/python/校外課程&自我學習/jieba/dict.txt.big.txt ...
Loading model from cache /var/folders/rc/x9l6wkn90dlgmkfy4d830zw80000gn/T/jieba.u9fe080374f2c8d462c27e757eba41adf.cache
Loading model cost 1.509 seconds.
Prefix dict has been built succesfully.


In [7]:
df['jieba_text'] = pd.DataFrame(jieba_text)[0]
df['jieba_text2'] = pd.DataFrame(jieba_text)[1]
df['jieba_count'] = pd.DataFrame(jieba_text)[2]

In [9]:
df.head(1)

,id,context,time,shares,likes,love,wow,haha,angry,sad,comments,likes_count,curator,hour,weekday,jieba_text,jieba_text2,jieba_count
0,124616330906800_1560501197318299,阿娘威！披羊皮的狼？竟大口嚼小雞\n#要打統編：小編真的是快嚇死了...😬😬\n\n影片來源...,2017-06-05 11:09:40,560,1144,11,757,21,292,86,589,2311,要打統編,11,Monday,"[阿娘, 威, 披, 羊皮, 狼, 竟大口, 嚼, 小雞, \n, 統編, 小編, 真的, ...",阿娘 威 披 羊皮 狼 竟大口 嚼 小雞 \n 統編 小編 真的 嚇死 \n \n \n ...,19


# TF-IDF演算法

In [10]:
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(df['jieba_text2'])

/Users/andy/anaconda/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [12]:
words = []
words2 = vectorizer.get_feature_names()
for i in tqdm_notebook(range(len(df['jieba_text2']))):
    temp_array = tfidf[i,:].toarray()
    for l in temp_array:
        words.append([(words2[x],l[x]) for x in (l*-1).argsort()][:5])

In [13]:
# 每篇文章的前五關鍵詞
df2 = pd.DataFrame(words,columns=['Keyword1','Keyword2','Keyword3','Keyword4','Keyword5'])
df2.head()

,Keyword1,Keyword2,Keyword3,Keyword4,Keyword5
0,"(草食性, 0.41657326701385844)","(竟大口, 0.41657326701385844)","(小雞, 0.3988213335189045)","(羊皮, 0.3988213335189045)","(阿娘, 0.33462266979698757)"
1,"(李毓芬, 0.5487664937801285)","(亮點, 0.48258358774415294)","(大落, 0.2865963176100915)","(演唱, 0.26571793531213106)","(昨天, 0.26571793531213106)"
2,"(心機, 0.6939407964222015)","(調虎離山, 0.5801917057019975)","(影帝, 0.25886033403402775)","(最萌, 0.23131359880740054)","(樂無編, 0.2201229708027141)"
3,"(感動, 0.32316350526125726)","(2roeqto, 0.2504423351739234)","(our, 0.2504423351739234)","(陪孕妻, 0.2504423351739234)","(living, 0.2504423351739234)"
4,"(畫家, 0.546678780884177)","(樓亡, 0.28957812644546865)","(要當, 0.2804329445867179)","(挑食, 0.2804329445867179)","(死神, 0.2733393904420885)"


In [16]:
df3 = pd.concat([df, df2], axis=1)
df3.head(1)

,id,context,time,shares,likes,love,wow,haha,angry,sad,...,hour,weekday,jieba_text,jieba_text2,jieba_count,Keyword1,Keyword2,Keyword3,Keyword4,Keyword5
0,124616330906800_1560501197318299,阿娘威！披羊皮的狼？竟大口嚼小雞\n#要打統編：小編真的是快嚇死了...😬😬\n\n影片來源...,2017-06-05 11:09:40,560,1144,11,757,21,292,86,...,11,Monday,"[阿娘, 威, 披, 羊皮, 狼, 竟大口, 嚼, 小雞, \n, 統編, 小編, 真的, ...",阿娘 威 披 羊皮 狼 竟大口 嚼 小雞 \n 統編 小編 真的 嚇死 \n \n \n ...,19,"(草食性, 0.41657326701385844)","(竟大口, 0.41657326701385844)","(小雞, 0.3988213335189045)","(羊皮, 0.3988213335189045)","(阿娘, 0.33462266979698757)"


# Word2Vec演算法

In [17]:
from gensim.models.word2vec import Word2Vec

建立模型

In [18]:
model = Word2Vec(df['jieba_text'])

In [19]:
def most_similar(w2v_model, words, topn=10):
    similar_df = pd.DataFrame()
    for word in words:
        try:
            similar_words = pd.DataFrame(w2v_model.wv.most_similar(word, topn=topn), columns=[word, 'cos'])
            similar_df = pd.concat([similar_df, similar_words], axis=1)
        except:
            print(word, "not found in Word2Vec model!")
    return similar_df

查看結果

In [20]:
most_similar(model, ['新聞', '體育', '娛樂', '政治', 'XDD', '小編', '夜市', '明星', '女孩'])

體育 not found in Word2Vec model!


/Users/andy/anaconda/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


,新聞,cos,娛樂,cos,政治,cos,XDD,cos,小編,cos,夜市,cos,明星,cos,女孩,cos
0,直播,0.959795,創意,0.998784,店家,0.997316,好笑,0.988211,好,0.988465,頭,0.999501,最強,0.999522,一種,0.998878
1,愛玩,0.959331,教育,0.998607,無罪,0.997233,撒嬌,0.987920,寶寶,0.980577,道路,0.999470,種,0.999458,惡整,0.998805
2,中心,0.956402,精神,0.998320,川習,0.996987,🤣,0.986974,吃,0.980428,玻璃,0.999293,秘訣,0.999422,甜蜜,0.998267
3,關鍵時刻,0.947280,啾,0.998245,國際,0.995791,脆嘀叔,0.986570,真的,0.977990,族,0.999280,童,0.999389,圍巾,0.998230
4,綜合,0.946602,街頭,0.998221,通緝犯,0.995274,哩厝編,0.986513,想,0.977289,閃,0.999240,超夯,0.999361,柴柴,0.998162
5,台,0.938506,結石,0.998053,衛生,0.995256,欸,0.985993,XDD,0.976551,登場,0.999238,免,0.999354,心,0.998019
6,朱姵慈,0.923990,健身,0.998023,號,0.994860,車部編,0.985809,惹,0.975322,鴨編,0.999196,謝,0.999348,抱,0.997945
7,反恐,0.922101,站立,0.998009,物流,0.994115,囂搞,0.985339,拍,0.973511,景點,0.999125,網紅,0.999271,偶,0.997754
8,吳念樺,0.921721,機,0.998004,網購,0.993992,太酷,0.984209,好笑,0.972431,電腦,0.999106,韓星,0.999258,翻譯,0.997728
9,李欣,0.918198,泰國,0.997993,火燒,0.993975,搶戲,0.983773,不敢相信,0.971960,國,0.999103,改造,0.999242,玩,0.997727


In [21]:
model_d250 = Word2Vec(df['jieba_text'], size=250, iter=10)
most_similar(model_d250, ['新聞', '體育', '娛樂', '政治', 'XDD', '小編', '夜市', '明星', '女孩'])

體育 not found in Word2Vec model!


/Users/andy/anaconda/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


,新聞,cos,娛樂,cos,政治,cos,XDD,cos,小編,cos,夜市,cos,明星,cos,女孩,cos
0,Nick,0.915449,Attribution,0.993436,模仿,0.985599,抱緊,0.955444,想,0.924509,線糊,0.979345,蝦米,0.996805,大姨媽,0.972272
1,節目,0.901065,金,0.993215,國際,0.976867,der,0.954049,好,0.885625,逢甲,0.979089,長髮,0.996341,汪汪,0.967014
2,房趣,0.891572,官網,0.993207,有害,0.974097,萌死,0.952110,感覺,0.880443,殺手,0.973239,周湯,0.995661,皇,0.965483
3,愛玩,0.882258,BibiNene,0.992933,歧視,0.966651,嘗試,0.951984,萌死,0.876297,小吃,0.972098,不想,0.995059,這招,0.965450
4,直播,0.873017,街頭,0.992042,恐攻,0.964786,牠,0.949918,媽媽,0.863194,拋,0.971702,張嘴,0.994988,卡哇伊,0.961905
5,挑,0.806437,色彩,0.991886,摩天輪,0.963963,愛,0.948467,可愛,0.858365,霸王餐,0.969875,凍齡,0.994573,瞬間,0.961377
6,中心,0.805555,內內,0.991526,加拿大,0.962654,😝,0.942588,分享,0.854629,螃蟹,0.969057,招讓,0.994515,挑戰,0.955900
7,調查,0.774872,Youtube,0.990872,攻擊,0.961540,眼睛,0.939250,真的,0.852909,味,0.968944,跟風,0.994318,怕水,0.955780
8,提醒您,0.770352,Keigo,0.990844,政府,0.960968,貓熊,0.939023,眼睛,0.850779,肉品,0.968859,鼻頭,0.994273,主人,0.953950
9,車,0.769464,海洋,0.990386,😫,0.959644,XDDDD,0.938075,太狂,0.849164,鯛,0.967305,電眼,0.994172,融化,0.952682


In [ ]:
model.save('word2vec.model')

In [ ]:
from gensim.models.word2vec import Word2Vec
model = Word2Vec.load('word2vec.model')